O objetivo deste trabalho (MVP - Engenharia de Dados) é responder as seguintes perguntas:
1) Quais são os filmes nos quais o diretor também é o escritor do filme?
2) Quais são os atores/atrizes que mais atuaram em filmes?
3) Quais são os filmes mais bem avaliados pelos usuários do IMDB para cada gênero?
4) Quais são os diretores dos filmes mais bem avaliados pelos usuários do IMDB?
5) Quais são as avaliações dos filmes pelos quais determinados atores são conhecidos?

Etapas do desenvolvimento do trabalho:
1) Escolha das perguntas definindo o objetivo do MVP;
2) Escolha da base de dados gratuita do site do IMDB para se atingir o objetivo definido do MVP;
3) Coleta e armazenamento dos dados na Plataforma Databricks (Camada Bronze);
4) Modelagem dos dados, elaboração do catálogo de dados, carga dos dados na Plataforma Databricks e verificação da qualidade dos dados (Camada Silver);
5) Análise dos dados e elaboração de consultas visando responder as perguntas feitas no objetivo do MVP (Camada Gold).

CAMADA BRONZE

Coleta dos dados no site do IMDB: https://datasets.imdbws.com/
Inicialmente, os dados foram armazenados em datasets do pandas. A seguir, foram armazenados em tabelas dentro do database db_bronze.

Importação dos dados das pessoas que trabalharam nos títulos (filmes, documentários, curtas, animações, séries)

In [0]:
import pandas as pd
import io
import requests

url = "https://datasets.imdbws.com/name.basics.tsv.gz"
dataset_pessoas = pd.read_csv(url, compression = 'gzip', sep = '\t')

dataset_pessoas.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


Importação dos dados dos títulos: tipo, título original, título promocional, booleano indicando se o filme é adulto ou não, ano de lançamento, fim (para séries), duração e gênero(s).

In [0]:
import pandas as pd
import io
import requests

url = "https://datasets.imdbws.com/title.basics.tsv.gz"
dataset_filmes = pd.read_csv(url, compression = 'gzip', sep = '\t')

dataset_filmes.head()

/root/.ipykernel/912/command-1448288978903677-3161710389:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_filmes = pd.read_csv(url, compression = 'gzip', sep = '\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


Importação dos dados das equipes que trabalharam nos títulos: diretores e escritores.

In [0]:
import pandas as pd
import io
import requests

url = "https://datasets.imdbws.com/title.crew.tsv.gz"
dataset_equipes = pd.read_csv(url, compression = 'gzip', sep = '\t')

dataset_equipes.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


Importação dos dados das avaliações dos títulos pelos usuários do site do IMDB, assim como o número de votos de cada título.

In [0]:
import pandas as pd
import io
import requests

url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
dataset_avaliacoes = pd.read_csv(url, compression = 'gzip', sep = '\t')

dataset_avaliacoes.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2064
1,tt0000002,5.6,279
2,tt0000003,6.5,2032
3,tt0000004,5.4,180
4,tt0000005,6.2,2798


Criando o database db_bronze

In [0]:
%sql CREATE DATABASE db_bronze;

Criando:
1) o dataframe df_pessoas com os dados do dataset_pessoas;
2) a tabela tb_pessoas_orig para armazenar os dados importados.


In [0]:
df_pessoas = spark.createDataFrame(dataset_pessoas)
df_pessoas.write.mode("overwrite").saveAsTable("db_bronze.tb_pessoas_orig")

In [0]:
%sql SELECT * FROM db_bronze.tb_pessoas_orig LIMIT 100

nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
nm0000006,Ingrid Bergman,1915,1982,"actress,producer,soundtrack","tt0034583,tt0036855,tt0038109,tt0038787"
nm0000007,Humphrey Bogart,1899,1957,"actor,producer,miscellaneous","tt0034583,tt0042593,tt0043265,tt0033870"
nm0000008,Marlon Brando,1924,2004,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0070849"
nm0000009,Richard Burton,1925,1984,"actor,producer,director","tt0061184,tt0087803,tt0059749,tt0057877"
nm0000010,James Cagney,1899,1986,"actor,director,producer","tt0029870,tt0031867,tt0042041,tt0055256"


Criando:
1) o dataframe df_filmes com os dados do dataset_filmes;
2) a tabela tb_filmes_orig para armazenar os dados importados.



In [0]:
df_filmes = spark.createDataFrame(dataset_filmes)
df_filmes.write.mode("overwrite").saveAsTable("db_bronze.tb_filmes_orig")

/databricks/spark/python/pyspark/sql/pandas/conversion.py:477: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert '0' with type str: tried to convert to int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [0]:
%sql SELECT * FROM db_bronze.tb_filmes_orig LIMIT 100

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


Criando:
1) o dataframe df_equipes com os dados do dataset_equipes;
2) a tabela tb_equipes_orig para armazenar os dados importados.

In [0]:
df_equipes = spark.createDataFrame(dataset_equipes)
df_equipes.write.mode("overwrite").saveAsTable("db_bronze.tb_equipes_orig")

In [0]:
%sql SELECT * FROM db_bronze.tb_equipes_orig LIMIT 100

tconst,directors,writers
tt0000001,nm0005690,\N
tt0000002,nm0721526,\N
tt0000003,nm0721526,\N
tt0000004,nm0721526,\N
tt0000005,nm0005690,\N
tt0000006,nm0005690,\N
tt0000007,"nm0005690,nm0374658",\N
tt0000008,nm0005690,\N
tt0000009,nm0085156,nm0085156
tt0000010,nm0525910,\N


Criando:
1) o dataframe df_avaliacoes com os dados do dataset_avaliacoes;
2) a tabela tb_avaliacoes_orig para armazenar os dados importados.

In [0]:
df_avaliacoes = spark.createDataFrame(dataset_avaliacoes)
df_avaliacoes.write.mode("overwrite").saveAsTable("db_bronze.tb_avaliacoes_orig")

In [0]:
%sql SELECT * FROM db_bronze.tb_avaliacoes_orig LIMIT 100

tconst,averageRating,numVotes
tt0000001,5.7,2064
tt0000002,5.6,279
tt0000003,6.5,2032
tt0000004,5.4,180
tt0000005,6.2,2798
tt0000006,5.1,190
tt0000007,5.4,877
tt0000008,5.4,2210
tt0000009,5.4,212
tt0000010,6.8,7624


CAMADA SILVER

Criando o database db_silver

In [0]:
%sql CREATE DATABASE db_silver

Criando o dataframe df_diretores apenas com os dados de diretores a partir do dataframe df_equipes.

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

df_diretores = df_equipes.select(df_equipes.tconst, explode(split(df_equipes.directors, ',')).alias("directors"))

df_diretores.show(10)

+---------+---------+
|   tconst|directors|
+---------+---------+
|tt0000001|nm0005690|
|tt0000002|nm0721526|
|tt0000003|nm0721526|
|tt0000004|nm0721526|
|tt0000005|nm0005690|
|tt0000006|nm0005690|
|tt0000007|nm0005690|
|tt0000007|nm0374658|
|tt0000008|nm0005690|
|tt0000009|nm0085156|
+---------+---------+
only showing top 10 rows



Criando a tabela tb_diretores com os dados do dataframe df_diretores

In [0]:
df_diretores.write.mode("overwrite").saveAsTable("db_silver.tb_diretores")

In [0]:
%sql SELECT * FROM db_silver.tb_diretores LIMIT 100

tconst,directors
tt0000001,nm0005690
tt0000002,nm0721526
tt0000003,nm0721526
tt0000004,nm0721526
tt0000005,nm0005690
tt0000006,nm0005690
tt0000007,nm0005690
tt0000007,nm0374658
tt0000008,nm0005690
tt0000009,nm0085156


Criando o dataframe df_escritores apenas com os dados de escritores a partir do dataframe df_equipes.

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

df_escritores = df_equipes.select(df_equipes.tconst, explode(split(df_equipes.writers, ',')).alias("writers"))

df_escritores.show(10)

+---------+---------+
|   tconst|  writers|
+---------+---------+
|tt0000001|       \N|
|tt0000002|       \N|
|tt0000003|       \N|
|tt0000004|       \N|
|tt0000005|       \N|
|tt0000006|       \N|
|tt0000007|       \N|
|tt0000008|       \N|
|tt0000009|nm0085156|
|tt0000010|       \N|
+---------+---------+
only showing top 10 rows



Criando a tabela tb_escritores com os dados do dataframe df_escritores.

In [0]:
df_escritores.write.mode("overwrite").saveAsTable("db_silver.tb_escritores")

In [0]:
%sql SELECT * FROM db_silver.tb_escritores LIMIT 100

tconst,writers
tt0000001,\N
tt0000002,\N
tt0000003,\N
tt0000004,\N
tt0000005,\N
tt0000006,\N
tt0000007,\N
tt0000008,\N
tt0000009,nm0085156
tt0000010,\N


Criando o dataframe df_pessoas_principal apenas com o id dos nomes (nconst) e os nomes das pessoas que trabalharam nos titulos.

In [0]:
df_pessoas_principal = df_pessoas.select(df_pessoas.nconst, df_pessoas.primaryName)
df_pessoas_principal.show(10)

+---------+---------------+
|   nconst|    primaryName|
+---------+---------------+
|nm0000001|   Fred Astaire|
|nm0000002|  Lauren Bacall|
|nm0000003|Brigitte Bardot|
|nm0000004|   John Belushi|
|nm0000005| Ingmar Bergman|
|nm0000006| Ingrid Bergman|
|nm0000007|Humphrey Bogart|
|nm0000008|  Marlon Brando|
|nm0000009| Richard Burton|
|nm0000010|   James Cagney|
+---------+---------------+
only showing top 10 rows



Criando a tabela tb_pessoas com os dados do dataframe df_pessoas_principal.

In [0]:
df_pessoas_principal.write.mode("overwrite").saveAsTable("db_silver.tb_pessoas")

In [0]:
%sql SELECT * FROM db_silver.tb_pessoas LIMIT 100

nconst,primaryName
nm0000001,Fred Astaire
nm0000002,Lauren Bacall
nm0000003,Brigitte Bardot
nm0000004,John Belushi
nm0000005,Ingmar Bergman
nm0000006,Ingrid Bergman
nm0000007,Humphrey Bogart
nm0000008,Marlon Brando
nm0000009,Richard Burton
nm0000010,James Cagney


Criando o dataframe df_pessoas_e_filmes que relaciona as pessoas aos filmes pelos quais ela é conhecida.

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

df_pessoas_e_filmes = df_pessoas.select(df_pessoas.nconst, explode(split(df_pessoas.knownForTitles, ',')).alias("tconst"))
df_pessoas_e_filmes.show(10)

+---------+---------+
|   nconst|   tconst|
+---------+---------+
|nm0000001|tt0072308|
|nm0000001|tt0050419|
|nm0000001|tt0053137|
|nm0000001|tt0027125|
|nm0000002|tt0037382|
|nm0000002|tt0075213|
|nm0000002|tt0117057|
|nm0000002|tt0038355|
|nm0000003|tt0057345|
|nm0000003|tt0049189|
+---------+---------+
only showing top 10 rows



Criando a tabela tb_pessoas_e_filmes com os dados do dataframe df_pessoas_e_filmes.

In [0]:
df_pessoas_e_filmes.write.mode("overwrite").saveAsTable("db_silver.tb_pessoas_e_filmes")

In [0]:
%sql SELECT * FROM db_silver.tb_pessoas_e_filmes LIMIT 100

nconst,tconst
nm0000001,tt0072308
nm0000001,tt0050419
nm0000001,tt0053137
nm0000001,tt0027125
nm0000002,tt0037382
nm0000002,tt0075213
nm0000002,tt0117057
nm0000002,tt0038355
nm0000003,tt0057345
nm0000003,tt0049189


Criando o dataframe df_filmes_principal apenas com o id do filme (tconst), tipo, titulo original, campo booleano dizendo se é adulto ou não e o ano de lançamento do titulo.

In [0]:
df_filmes_principal = df_filmes.select(df_filmes.tconst, df_filmes.titleType, df_filmes.originalTitle, df_filmes.isAdult, df_filmes.startYear)
df_filmes_principal.show(10)

+---------+---------+--------------------+-------+---------+
|   tconst|titleType|       originalTitle|isAdult|startYear|
+---------+---------+--------------------+-------+---------+
|tt0000001|    short|          Carmencita|      0|     1894|
|tt0000002|    short|Le clown et ses c...|      0|     1892|
|tt0000003|    short|      Pauvre Pierrot|      0|     1892|
|tt0000004|    short|         Un bon bock|      0|     1892|
|tt0000005|    short|    Blacksmith Scene|      0|     1893|
|tt0000006|    short|   Chinese Opium Den|      0|     1894|
|tt0000007|    short|Corbett and Court...|      0|     1894|
|tt0000008|    short|Edison Kinetoscop...|      0|     1894|
|tt0000009|    movie|          Miss Jerry|      0|     1894|
|tt0000010|    short|La sortie de l'us...|      0|     1895|
+---------+---------+--------------------+-------+---------+
only showing top 10 rows



Criando a tabela tb_filmes com os dados do dataframe df_filmes_principal.

In [0]:
df_filmes_principal.write.mode("overwrite").saveAsTable("db_silver.tb_filmes")

In [0]:
%sql SELECT * FROM db_silver.tb_filmes LIMIT 100

tconst,titleType,originalTitle,isAdult,startYear
tt0000001,short,Carmencita,0,1894
tt0000002,short,Le clown et ses chiens,0,1892
tt0000003,short,Pauvre Pierrot,0,1892
tt0000004,short,Un bon bock,0,1892
tt0000005,short,Blacksmith Scene,0,1893
tt0000006,short,Chinese Opium Den,0,1894
tt0000007,short,Corbett and Courtney Before the Kinetograph,0,1894
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,0,1894
tt0000009,movie,Miss Jerry,0,1894
tt0000010,short,La sortie de l'usine Lumière à Lyon,0,1895


Criando o dataframe df_filmes_e_generos que relaciona os titulos aos seus generos.

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

df_filmes_e_generos = df_filmes.select(df_filmes.tconst, explode(split(df_filmes.genres, ',')).alias("genres"))
df_filmes_e_generos.show(10)

+---------+-----------+
|   tconst|     genres|
+---------+-----------+
|tt0000001|Documentary|
|tt0000001|      Short|
|tt0000002|  Animation|
|tt0000002|      Short|
|tt0000003|  Animation|
|tt0000003|     Comedy|
|tt0000003|    Romance|
|tt0000004|  Animation|
|tt0000004|      Short|
|tt0000005|     Comedy|
+---------+-----------+
only showing top 10 rows



Criando a tabela tb_filmes_e_generos com os dados do dataframe df_filmes_e_generos

In [0]:
df_filmes_e_generos.write.mode("overwrite").saveAsTable("db_silver.tb_filmes_e_generos")

In [0]:
%sql SELECT * FROM db_silver.tb_filmes_e_generos LIMIT 100

tconst,genres
tt0000001,Documentary
tt0000001,Short
tt0000002,Animation
tt0000002,Short
tt0000003,Animation
tt0000003,Comedy
tt0000003,Romance
tt0000004,Animation
tt0000004,Short
tt0000005,Comedy


Removendo as linhas correspondentes aos filmes adultos e títulos que não são filmes da tabela tb_filmes.

In [0]:
%sql DELETE FROM db_silver.tb_filmes WHERE isAdult = 1 OR titleType <> 'movie'

num_affected_rows
10246290


Verificando se a remoção foi bem sucedida

In [0]:
%sql SELECT * FROM db_silver.tb_filmes WHERE isAdult >= 1

tconst,titleType,originalTitle,isAdult,startYear


In [0]:
%sql SELECT * FROM db_silver.tb_filmes WHERE titleType <> 'movie'

tconst,titleType,originalTitle,isAdult,startYear


In [0]:
%sql SELECT count(*) AS TOTAL FROM db_silver.tb_filmes

TOTAL
676472


Verificando a qualidade dos dados 

In [0]:
%sql SELECT * FROM db_silver.tb_pessoas WHERE primaryName = '\\N'

nconst,primaryName
nm12641157,\N
nm15245498,\N
nm14074784,\N
nm14075045,\N
nm10019610,\N
nm13220364,\N
nm2505107,\N
nm12910234,\N
nm14936186,\N
nm15416513,\N


Removendo os nomes em branco ou nulos

In [0]:
%sql DELETE FROM db_silver.tb_pessoas WHERE primaryName = '\\N'

num_affected_rows
50


In [0]:
%sql SELECT * FROM db_silver.tb_diretores WHERE directors = '\\N'

tconst,directors
tt0111059,\N
tt0111242,\N
tt0111251,\N
tt0111315,\N
tt0111322,\N
tt0111380,\N
tt0111433,\N
tt0111468,\N
tt0111529,\N
tt0111576,\N


Removendo os nomes em branco ou nulos

In [0]:
%sql DELETE FROM db_silver.tb_diretores WHERE directors = '\\N'

num_affected_rows
3875767


In [0]:
%sql SELECT * FROM db_silver.tb_escritores WHERE writers = '\\N'

tconst,writers
tt0080762,\N
tt0080763,\N
tt0080775,\N
tt0080776,\N
tt0080807,\N
tt0080818,\N
tt0080829,\N
tt0080875,\N
tt0080876,\N
tt0080877,\N


Removendo os nomes em branco ou nulos

In [0]:
%sql DELETE FROM db_silver.tb_escritores WHERE writers = '\\N'

num_affected_rows
4511014


Verificando se as remoções foram bem sucedidas

In [0]:
%sql SELECT * FROM db_silver.tb_filmes WHERE originalTitle = '\\N'

tconst,titleType,originalTitle,isAdult,startYear


In [0]:
%sql SELECT * FROM db_silver.tb_filmes WHERE startYear <= 0

tconst,titleType,originalTitle,isAdult,startYear


Criando a tabela tb_avaliacoes com os dados do dataframe df_avaliacoes

In [0]:
df_avaliacoes.write.mode("overwrite").saveAsTable("db_silver.tb_avaliacoes")
df_avaliacoes.show(10)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2064|
|tt0000002|          5.6|     279|
|tt0000003|          6.5|    2032|
|tt0000004|          5.4|     180|
|tt0000005|          6.2|    2798|
|tt0000006|          5.1|     190|
|tt0000007|          5.4|     877|
|tt0000008|          5.4|    2210|
|tt0000009|          5.4|     212|
|tt0000010|          6.8|    7624|
+---------+-------------+--------+
only showing top 10 rows



In [0]:
%sql SELECT * FROM db_silver.tb_avaliacoes LIMIT 100

tconst,averageRating,numVotes
tt0000001,5.7,2064
tt0000002,5.6,279
tt0000003,6.5,2032
tt0000004,5.4,180
tt0000005,6.2,2798
tt0000006,5.1,190
tt0000007,5.4,877
tt0000008,5.4,2210
tt0000009,5.4,212
tt0000010,6.8,7624


Verificando a qualidade dos dados na tabela tb_avaliacoes

In [0]:
%sql SELECT * FROM db_silver.tb_avaliacoes WHERE averageRating < 0 OR averageRating > 10

tconst,averageRating,numVotes


In [0]:
%sql SELECT * FROM db_silver.tb_avaliacoes WHERE numVotes <= 0

tconst,averageRating,numVotes


CAMADA GOLD

Criando o database db_gold

In [0]:
%sql CREATE DATABASE db_gold

Criando a tabela tb_diretores_e_escritores na qual constam as pessoas que atuaram como diretor e escritor em títulos.

In [0]:
%sql CREATE TABLE db_gold.tb_diretores_e_escritores
AS SELECT D.tconst AS id_filme, D.directors AS id_pessoa 
FROM db_silver.tb_diretores AS D, db_silver.tb_escritores AS E
WHERE D.directors = E.writers AND D.tconst = E.tconst

num_affected_rows,num_inserted_rows


In [0]:
%sql SELECT * FROM db_gold.tb_diretores_e_escritores LIMIT 100

id_filme,id_pessoa
tt0050986,nm0000005
tt0064897,nm0000005
tt0074147,nm0000005
tt0091327,nm0000005
tt0111204,nm0000005
tt2297772,nm0000005
tt6720536,nm0000005
tt6725158,nm0000005
tt0070642,nm0000018
tt0044000,nm0000019


Selecão com os nomes das pessoas que atuaram como diretor e escritor e os respectivos filmes

In [0]:
%sql SELECT F.originalTitle AS Titulo_Original, P.primaryName AS Nome_Pessoa
FROM db_silver.tb_filmes AS F, db_silver.tb_pessoas AS P, db_gold.tb_diretores_e_escritores AS D
WHERE F.tconst = D.id_filme AND P.nconst = D.id_pessoa

Titulo_Original,Nome_Pessoa
Chaddabeshi,Ajitava Barat
Chup,Ajitava Barat
Bhoote Biswas Koren?,Ajitava Barat
Ekti Raat,Ajitava Barat
Les chroniques de l'île du roi boiteux (n°1: Pâques. n°2: Les enfants du serpent),Jérôme Bouyer
Vivants chez les morts,Jérôme Bouyer
Au fil de l'eau,Jérôme Bouyer
"Goodbye, Butterfly",Tyler Wayne
Frauenzimmer,Saara Waasner
Das Kreuz mit der Liebe,Saara Waasner


Criando a tabela tb_filmes_e_avaliacoes com os filmes mais bem avaliados pelos usuários do IMDB e que receberam 500000 votos ou mais.

In [0]:
%sql CREATE TABLE db_gold.tb_filmes_e_avaliacoes
SELECT F.tconst AS id_filme, F.originalTitle AS titulo_original, F.startYear AS ano, G.genres AS genero, A.averageRating AS avaliacao, A.numVotes AS num_votos
FROM db_silver.tb_filmes AS F, db_silver.tb_filmes_e_generos AS G, db_silver.tb_avaliacoes AS A
WHERE F.tconst = G.tconst AND A.tconst = F.tconst AND A.numVotes >=500000
ORDER BY A.averageRating DESC
--LIMIT 500

num_affected_rows,num_inserted_rows


In [0]:
%sql SELECT * FROM db_gold.tb_filmes_e_avaliacoes 

id_filme,titulo_original,ano,genero,avaliacao,num_votos
tt0111161,The Shawshank Redemption,1994,Drama,9.3,2914087
tt0068646,The Godfather,1972,Crime,9.2,2030748
tt0068646,The Godfather,1972,Drama,9.2,2030748
tt0108052,Schindler's List,1993,Biography,9.0,1462856
tt0108052,Schindler's List,1993,Drama,9.0,1462856
tt0108052,Schindler's List,1993,History,9.0,1462856
tt0071562,The Godfather Part II,1974,Crime,9.0,1374762
tt0071562,The Godfather Part II,1974,Drama,9.0,1374762
tt0167260,The Lord of the Rings: The Return of the King,2003,Action,9.0,1995275
tt0167260,The Lord of the Rings: The Return of the King,2003,Adventure,9.0,1995275


Seleção dos diretores dos filmes mais bem avaliados pelos usuários do IMDB

In [0]:
%sql SELECT DISTINCT F.titulo_original, F.ano, F.avaliacao, F.num_votos, P.primaryName AS diretor
FROM db_silver.tb_diretores AS D, db_gold.tb_filmes_e_avaliacoes AS F, db_silver.tb_pessoas AS P
WHERE D.tconst = F.id_filme AND P.nconst = D.directors
ORDER BY F.avaliacao DESC

titulo_original,ano,avaliacao,num_votos,diretor
The Shawshank Redemption,1994,9.3,2914087,Frank Darabont
The Godfather,1972,9.2,2030748,Francis Ford Coppola
The Godfather Part II,1974,9.0,1374762,Francis Ford Coppola
Schindler's List,1993,9.0,1462856,Steven Spielberg
The Lord of the Rings: The Return of the King,2003,9.0,1995275,Peter Jackson
12 Angry Men,1957,9.0,874132,Sidney Lumet
The Dark Knight,2008,9.0,2895365,Christopher Nolan
Pulp Fiction,1994,8.9,2239616,Quentin Tarantino
The Lord of the Rings: The Fellowship of the Ring,2001,8.9,2023895,Peter Jackson
"Il buono, il brutto, il cattivo",1966,8.8,817280,Sergio Leone


Seleção com as avaliações dos filmes pelos quais Daniel Craig é conhecido

In [0]:
%sql SELECT DISTINCT F.titulo_original, F.ano, F.avaliacao, F.num_votos, P.primaryName AS pessoa
FROM db_gold.tb_filmes_e_avaliacoes AS F, db_silver.tb_pessoas AS P, db_silver.tb_pessoas_e_filmes AS E
WHERE E.tconst = F.id_filme AND P.nconst = E.nconst AND P.primaryName = 'Daniel Craig'
ORDER BY F.avaliacao DESC

titulo_original,ano,avaliacao,num_votos,pessoa
Casino Royale,2006,8.0,698373,Daniel Craig
Skyfall,2012,7.8,734632,Daniel Craig


In [0]:
%sql SELECT DISTINCT F.titulo_original, F.ano, F.avaliacao, F.num_votos, P.primaryName AS pessoa
FROM db_gold.tb_filmes_e_avaliacoes AS F, db_silver.tb_pessoas AS P, db_silver.tb_pessoas_e_filmes AS E
WHERE E.tconst = F.id_filme AND P.nconst = E.nconst AND P.primaryName = 'Cate Blanchett'
ORDER BY F.avaliacao DESC

titulo_original,ano,avaliacao,num_votos,pessoa
The Lord of the Rings: The Fellowship of the Ring,2001,8.9,2023895,Cate Blanchett
The Curious Case of Benjamin Button,2008,7.8,700278,Cate Blanchett


In [0]:
%sql SELECT DISTINCT F.titulo_original, F.ano, F.avaliacao, F.num_votos, P.primaryName AS pessoa
FROM db_gold.tb_filmes_e_avaliacoes AS F, db_silver.tb_pessoas AS P, db_silver.tb_pessoas_e_filmes AS E
WHERE E.tconst = F.id_filme AND P.nconst = E.nconst AND P.primaryName = 'Keanu Reeves'
ORDER BY F.avaliacao DESC

titulo_original,ano,avaliacao,num_votos,pessoa
The Matrix,1999,8.7,2070858,Keanu Reeves
The Matrix Reloaded,2003,7.2,634692,Keanu Reeves


Seleção dos filmes de ação mais bem avaliados pelos usuários do IMDB

In [0]:
%sql SELECT titulo_original, ano, avaliacao, num_votos 
FROM db_gold.tb_filmes_e_avaliacoes
WHERE genero = 'Action'

titulo_original,ano,avaliacao,num_votos
The Lord of the Rings: The Return of the King,2003,9.0,1995275
The Dark Knight,2008,9.0,2895365
The Lord of the Rings: The Fellowship of the Ring,2001,8.9,2023895
Inception,2010,8.8,2572007
The Lord of the Rings: The Two Towers,2002,8.8,1798828
Star Wars: Episode V - The Empire Strikes Back,1980,8.7,1390832
The Matrix,1999,8.7,2070858
Star Wars,1977,8.6,1460451
Terminator 2: Judgment Day,1991,8.6,1183145
Gladiator,2000,8.5,1635511


In [0]:
%sql SELECT titulo_original, ano, avaliacao, num_votos 
FROM db_gold.tb_filmes_e_avaliacoes
WHERE genero = 'Sci-Fi'

titulo_original,ano,avaliacao,num_votos
Inception,2010,8.8,2572007
Interstellar,2014,8.7,2128915
The Matrix,1999,8.7,2070858
Terminator 2: Judgment Day,1991,8.6,1183145
Back to the Future,1985,8.5,1318729
The Prestige,2006,8.5,1452865
Alien,1979,8.5,959581
Avengers: Infinity War,2018,8.4,1217273
Aliens,1986,8.4,768853
2001: A Space Odyssey,1968,8.3,724830


Seleção dos filmes de guerra mais bem avaliados pelos usuarios do IMDB

In [0]:
%sql SELECT titulo_original, ano, avaliacao, num_votos 
FROM db_gold.tb_filmes_e_avaliacoes
WHERE genero = 'War'

titulo_original,ano,avaliacao,num_votos
Saving Private Ryan,1998,8.6,1509055
Casablanca,1942,8.5,609824
Inglourious Basterds,2009,8.4,1599875
Apocalypse Now,1979,8.4,715352
Braveheart,1995,8.3,1097528
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,1964,8.3,521241
Full Metal Jacket,1987,8.3,795545
El laberinto del fauno,2006,8.2,706487
Fury,2014,7.6,552828


Seleção dos filmes de drama mais bem avaliados pelos usuarios do IMDB

In [0]:
%sql SELECT titulo_original, ano, avaliacao, num_votos 
FROM db_gold.tb_filmes_e_avaliacoes
WHERE genero = 'Drama'

titulo_original,ano,avaliacao,num_votos
The Shawshank Redemption,1994,9.3,2914087
The Godfather,1972,9.2,2030748
Schindler's List,1993,9.0,1462856
The Godfather Part II,1974,9.0,1374762
The Lord of the Rings: The Return of the King,2003,9.0,1995275
12 Angry Men,1957,9.0,874132
The Dark Knight,2008,9.0,2895365
The Lord of the Rings: The Fellowship of the Ring,2001,8.9,2023895
Pulp Fiction,1994,8.9,2239616
"Il buono, il brutto, il cattivo",1966,8.8,817280


Autoavaliação:
O trabalho foi desenvolvido da melhor forma possível de acordo com o tempo disponível. A pergunta 2 não foi respondida porque não foi possivel baixar os dados da tabela referente aos atores/atrizes, que é muito grande para ser utilizada no Comunity Databricks